# Coding Assignment - 4

#### Authors:
+ Yogananth Mahalingam - ym24 - MCS-DS
+ Qi Zhou - qizhou8 - MCS-DS
+ Derek Zhang - derekz3 - MCS-DS

#### Contributions:
Yoga, Derek & Qi worked on it independently. Brainstormed together & created the final version.

## Part 2: HMM

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from matplotlib.colors import LogNorm



In [2]:
itmax = 100

mx = 3
mz = 2
w = np.array([0.5, 0.5])
A = np.array([[0.5, 0.5], 
              [0.5, 0.5]])
B = np.array([[1/9, 3/9, 5/9], 
              [1/6, 2/6, 3/6]])

X = np.loadtxt("coding4_part2_data.txt").astype(int)

Z_test = np.genfromtxt("Coding4_part2_Z.txt").astype(int)


In [3]:
def BW_onstep(X, mx, mz, w, A, B):
    new_w = w
    new_A = A
    new_B = B
    
    alpha = calculate_alpha(X, mx, mz, w, new_A, new_B)
    beta = calculate_beta(X, mx, mz, w, new_A, new_B)
    gamma = calculate_gamma(X, mx, mz, w, A, B, alpha, beta)
    
    new_w = update_w(X, mx, mz, w, A, B, alpha, beta, gamma)
    new_A = update_A(X, mx, mz, w, A, B, alpha, beta, gamma)
    new_B = update_B(X, mx, mz, w, A, B, alpha, beta, gamma)
    
    return (new_w, new_A, new_B)

In [4]:
def calculate_alpha(X, mx, mz, w, A, B):
    n = X.shape[0]
    alpha = np.zeros((n, mz))
    for t in range(n):
        for i in range(mz):
            x = X[t] - 1
            if t == 0:
                #print(f"B[i, :]={B[i, :]}, x={x}")
                alpha[t,i]=w[i] * B[i,x]
            else:
                t_prev = t-1
                alpha[t,i] = np.sum(alpha[t-1, :] * A[:, i] * B[i, X[t]-1])
    return alpha

alpha = calculate_alpha(X, mx, mz, w, A, B)


In [5]:
def calculate_beta(X, mx, mz, w, A, B):
    n = X.shape[0]
    beta = np.ones((n, mz))
    
    for t in reversed(range(n-1)):
        
        for i in range(mz):
            t_next = t+1
            x = X[t_next]-1
            
            beta[t,i] = np.sum( A[i, :] * B[:, X[t+1]-1 ] * beta[t+1, :] )

    
    return beta

beta = calculate_beta(X, mx, mz, w, A, B)


In [6]:
def calculate_gamma(X, mx, mz, w, A, B, alpha, beta):
    n = X.shape[0]
    gamma = np.zeros((n-1, mz, mz))
    
    for t in range(n-1):
        for i in range(mz):
            for j in range(mz):
                t_next = t+1
                x = X[t_next]-1

                gamma[t, i, j] = alpha[t,i] * A[i,j] * B[j, X[t+1]-1] * beta[t+1, j]
    return gamma

gamma = calculate_gamma(X, mx, mz, w, A, B, alpha, beta)
    

In [7]:
def update_w(X, mx, mz, w, A, B, alpha, beta, gamma):
    updated_w = np.sum(gamma[0], axis=1)
    return updated_w

updated_w = update_w(X, mx, mz, w, A, B, alpha, beta, gamma)


In [8]:
def update_A(X, mx, mz, w, A, B, alpha, beta, gamma):
    updated_A = A
    updated_A = np.sum(gamma, axis=0) / np.sum(np.sum(gamma, axis=2), axis=0).reshape(-1,1)
    return updated_A

updated_A = update_A(X, mx, mz, w, A, B, alpha, beta, gamma)


In [9]:
# def update_A(X, mx, mz, w, A, B, alpha, beta, gamma):
#     updated_A = np.zeros(A.shape)
#     for i in range(mx):
#         for i in range(mx):
#     updated_A = np.sum(gamma, axis=0) / np.sum(np.sum(gamma, axis=2), axis=0).reshape(-1,1)
#     return updated_A

# updated_A = update_A(X, mx, mz, w, A, B, alpha, beta, gamma)


In [10]:
def update_B(X, mx, mz, w, A, B, alpha, beta, gamma):
    n = X.shape[0]
    updated_B = np.zeros(B.shape)
    
    gamma_t = np.sum(gamma, axis=1)
    gamma_n = np.sum(gamma[n-2], axis=0)
    gamma_t = np.append(gamma_t, np.array([gamma_n]), axis=0)
    
    for i in range(mz):
        for l in range(mx):
            
            gamma_t_matched = gamma_t[X==(l+1)] 
            nr = np.sum(gamma_t_matched, axis = 0)[i]
            dr = np.sum(gamma_t, axis = 0)[i]
            
            updated_B[i,l] = nr / dr
    return updated_B

updated_B = update_B(X, mx, mz, w, A, B, alpha, beta, gamma)


In [11]:
def myBW(X, mx, mz, w, A, B, itmax):
    new_A = A
    new_B = B
    for i in range(itmax):
        (new_w, new_A, new_B) = BW_onstep(X, mx, mz, w, new_A, new_B)
        
    
    return (new_w, new_A, new_B)
    

In [12]:
(new_w, new_A, new_B) = myBW(X, mx, mz, w, A, B, itmax)
print(f"new_A" )
print(f"{new_A}" )
print(f"new_B:" )
print(f"{new_B}" )

new_A
[[0.48546086 0.51453914]
 [0.48502678 0.51497322]]
new_B:
[[0.28499999 0.19000006 0.52499995]
 [0.28500001 0.18999994 0.52500004]]


In [13]:
def calculate_delta(X, mx, mz, w, A, B):
    n = X.shape[0]
    delta = np.zeros((n, mz))
    for t in range(n):
        for i in range(mz):
            x = X[t] - 1
            if t == 0:
                #print(f"B[i, :]={B[i, :]}, x={x}")
                delta[t,i]=w[i] * B[i,x]
            else:
                delta[t,i] = np.max(delta[t-1, :] * A[:, i]) * B[i, X[t]-1]
                #delta[t,i] = np.sum(delta[t-1, :] * A[:, i] * B[i, X[t]-1])
    return delta

# delta = calculate_delta(X, mx, mz, w, new_A_Viterbi, new_B_Viterbi)
# delta


In [14]:
def myViterbi(X, mx, mz, w, A, B):
    n = X.shape[0]
    Z = np.zeros(n)
    
    delta = calculate_delta(X, mx, mz, w, A, B)
    for t in reversed(range(n)):
        if t == n-1:
            index = np.argmax(delta[t])
            Z[t] = index + 1
        else:
            j = int(Z[t+1] - 1)
            #print(A[:, j])
            index = np.argmax(delta[t] * A[:, j])
            Z[t] = index + 1
            
    
    return Z


In [15]:
new_A_Viterbi = np.array([
    [0.49793938, 0.50206062],
    [0.44883431, 0.55116569]
])

new_B_Viterbi = np.array([
    [0.22159897, 0.20266127, 0.57573976],
    [0.34175148, 0.17866665, 0.47958186]
])

In [16]:
Z = myViterbi(X, mx, mz, w, new_A_Viterbi, new_B_Viterbi)
np.array_equal(Z, Z_test)

True